In [92]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

# Let's get RouteID/RouteName information from web

In [93]:
URL = 'https://www.miamidade.gov/transportation-publicworks/routes.asp'
page = requests.get(URL)

routes_name = BeautifulSoup(page.content)

C:\Users\andy\Anaconda3\lib\site-packages\bs4\__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file C:\Users\andy\Anaconda3\lib\runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))


In [94]:
route_ids = []
for i in range(len(routes_name.find_all('option'))):
    route_ids.append(str(routes_name.find_all('option')[i]).split('>')[0].split('value=')[1].replace('"','').replace('#',''))
    
route_names = []
for i in range(len(routes_name.find_all('option'))):
    route_names.append(routes_name.find_all('option')[i].contents[0])

In [95]:
route_names = pd.concat([pd.DataFrame(route_ids[1:]), pd.DataFrame(route_names[1:])], axis=1)
route_names.columns = ['RouteID','RouteName']

In [96]:
route_names['RouteID'] = route_names['RouteID'].astype('int')

In [97]:
# sort values by routeid
route_names = route_names.sort_values(by='RouteID')

In [98]:
URL2 = 'http://www.miamidade.gov/transit/WebServices/BusRouteDirections/?RouteID'
page2 = requests.get(URL2)

routes_info = BeautifulSoup(page2.content)
routes_info = routes_info.find_all('routeid')


routes_list = []
for i in range(len(routes_info)):
    routes_list.append(int(routes_info[i].contents[0]))
    
routes_list = list(routes_list)

C:\Users\andy\Anaconda3\lib\site-packages\bs4\__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file C:\Users\andy\Anaconda3\lib\runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))


In [99]:
direction_info = BeautifulSoup(page2.content)
direction_info = direction_info.find_all('direction')

dir_list = []
for i in range(len(direction_info)):
    dir_list.append(direction_info[i].contents[0])
    


C:\Users\andy\Anaconda3\lib\site-packages\bs4\__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file C:\Users\andy\Anaconda3\lib\runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))


In [100]:
routes_dir_list = pd.concat([pd.DataFrame(routes_list),pd.DataFrame(dir_list)], axis=1)
routes_dir_list.columns = ['RouteID','Direction']

In [101]:
routes_dir_list = routes_dir_list.sort_values(by = 'RouteID')

In [102]:
routes_dir_list = routes_dir_list.merge(route_names, on = 'RouteID')

In [103]:
# set route list as unique routeIDs for script below
routes_list = routes_dir_list['RouteID'].unique().tolist()

# Let's get stop information from web-scraping

In [104]:
#route = 77
def stop_df_webscraping(routes_list):


    for route in routes_list:
        ## Looks like some routes, such as 39 is missing so just pass those routes
        try:
            URL = 'https://www.miamidade.gov/transit/mobile/appPages/MapBusStop/?RouteID='+str(route)+'&amp;Dir=&amp;StopID=&amp;Sequence=&amp;nearby=&amp;BusID=ALL&amp;src=mobile'
            page = requests.get(URL)

            soup = BeautifulSoup(page.content)
            result = soup.find_all('script')

            test = result[1].contents[0].split(';')
            df = pd.DataFrame(test)
            stops = df[df[0].str.contains('StopAttributes')]
            stops = pd.DataFrame(stops[0].str.split('=',1).tolist(),
                                             columns = ['flips','row'])

            stops['clean'] = stops['row'].apply(lambda x:x.replace('\r','').replace('\n',''))
            stops['final'] = stops['clean'].apply(lambda x:x.replace('{','').replace('}','').replace('\'',''))

            final = stops[0::2]
            final2 = pd.DataFrame(final['final'].str.split(',').tolist())

            for i in range(len(final2.columns)):
                pd.DataFrame(final2[i].str.split(':').tolist())

            final3 = [pd.DataFrame(final2[i].str.split(':').tolist()) for i in range(len(final2.columns))]
            pdList = [final3[i] for i in range(len(final3))]

            # List of your dataframes
            new_df = pd.concat(pdList, axis=1)
            new_col = list(new_df[0].values[0])
            new_df = new_df[1]
            new_df.columns = new_col

            return new_df
        
        except:
            pass


In [105]:
pdList = []
for route in routes_list:
    pdList.append(stop_df_webscraping([route]))


full_df = pd.concat(pdList)

C:\Users\andy\Anaconda3\lib\site-packages\bs4\__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file C:\Users\andy\Anaconda3\lib\runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))


In [106]:
len(full_df.RouteID.unique())

120

# Let's get polyline (route multi-line) from webscraping

since some of the routes are missing from Open Hub API, let's try to get routes from web scraping

In [107]:
def rt_df_webscraping(routes_list):


    for route in routes_list:
        ## Looks like some routes, such as 39 is missing so just pass those routes
        try:

            URL = 'https://www.miamidade.gov/transit/mobile/appPages/MapBusStop/?RouteID='+str(route)+'&amp;Dir=&amp;StopID=&amp;Sequence=&amp;nearby=&amp;BusID=ALL&amp;src=mobile'
            page = requests.get(URL)

            soup = BeautifulSoup(page.content)
            result = soup.find_all('script')

            test = result[1].contents[0].split(';')
            df = pd.DataFrame(test)

            routes = df[df[0].str.contains('polyline')]
            routes = pd.DataFrame(routes[0].str.split('=',1).tolist(),columns = ['flips','row'])
            #routes

            #routes['clean'] = routes['row'].apply(lambda x:x.replace('\r','').replace('\n',''))

            clean = []

            for i in routes['row']:
                try:
                    clean.append(i.replace('\r','').replace('\n',''))
                except:
                    pass

            #routes['final'] = routes['clean'].apply(lambda x:x.replace('{','').replace('}','').replace('\'',''))

            final = []

            for i in clean:
                try:
                    final.append(i.replace('\r','').replace('\n',''))
                except:
                    pass

            final = pd.DataFrame(final)[0::2]
            final = final.reset_index()
            
            for i in range(len(final)):
                final[0] = final[0].apply(lambda x:x.replace('{','').replace(' type: "polyline"','').replace(',paths: ','').replace('}',''))
                
            final.columns = ['index','coord']
            final['RouteID'] = route
            
            # remove coord that is empty    
            for coord in range(len(final['coord'][final['RouteID'] == route])):
                if len(final['coord'][final['RouteID'] == route][coord]) == 2:
                    final.drop(coord, inplace = True)

            final.reset_index(inplace = True, drop = True)

            return final
        
        except:
            pass

In [108]:
pdList2 = []

# routes_list = [11,51,77,277,119,120]  # If you want to select only 6 routes, use this. Other wise, ignore this
routes_list = [11,51,77,277,119,120,112,9,27,3,8,22] # If you want to select only 6 routes plus expanding routes, use this. Other wise, ignore this
routes_list = sorted(routes_list)  # Same as above

for route in routes_list:
    pdList2.append(rt_df_webscraping([route]))


full_df2 = pd.concat(pdList2)

C:\Users\andy\Anaconda3\lib\site-packages\bs4\__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file C:\Users\andy\Anaconda3\lib\runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))


In [109]:
full_df2 = full_df2.reset_index()
full_df2.index = full_df2['level_0']

In [110]:
def feature_route_webscrap(route):
    feature_rt = {'type':'Feature',
                   'properties':{},
                   'geometry':{'type':'MultiLineString',
                               'coordinates':[]}}
    
    feature_rt['properties'] = {'RouteID': route}
    for i in range(len(full_df2[full_df2['RouteID']==route])):
        feature_rt['geometry']['coordinates'].append(full_df2[full_df2['RouteID']==route]['coord'][i])
    
    return feature_rt

In [111]:
## Error here, coord at index 9 is missing for some reason. Just ignore this part - it still works fine; I guess tis mistake by MDC.
#full_df2[full_df2['RouteID'] == 112]
#full_df2['coord'][full_df2['RouteID'] == 112][9]

### Let's make geojson version of routes from web-scraping

In [112]:
#first = routes_list[0]
#routes_selected = routes_list[1:]
feature_col = {'type':'FeatureCollection', 'features':[]}

for route in routes_list:
    #for example, RouteID == 81 does not have any coordinate since this route is currently out of service
    if len(full_df2[full_df2['RouteID']==route]) == 0:
        continue
    else:        
    #feature_route_webscrap(first).append(feature_route_webscrap(i))
        feature_col['features'].append(feature_route_webscrap(route))

### color codes

In [113]:
color_code = {
 # "aliceblue": "#f0f8ff",
 # "antiquewhite": "#faebd7",
 # "aqua": "#00ffff",
 # "aquamarine": "#7fffd4",
  "azure": "#f0ffff",
   "brown": "#a52a2a",
  "bisque": "#ffe4c4",
  "black": "#000000",
  "darkgrey": "#a9a9a9",
  "blue": "#0000ff",
  "blueviolet": "#8a2be2",
  "beige": "#f5f5dc",
    "blanchedalmond": "#ffebcd",
  "burlywood": "#deb887",
  "cadetblue": "#5f9ea0",
  "chartreuse": "#7fff00",
  "chocolate": "#d2691e",
  "coral": "#ff7f50",
  "cornflowerblue": "#6495ed",
  "cornsilk": "#fff8dc",
  "crimson": "#dc143c",
  "cyan": "#00ffff",
  "darkblue": "#00008b",
  "darkcyan": "#008b8b",
  "darkgoldenrod": "#b8860b",
  "darkgray": "#a9a9a9",
  "darkgreen": "#006400",
  "darkkhaki": "#bdb76b",
  "darkmagenta": "#8b008b",
  "darkolivegreen": "#556b2f",
  "darkorange": "#ff8c00",
  "darkorchid": "#9932cc",
  "darkred": "#8b0000",
  "darksalmon": "#e9967a",
  "darkseagreen": "#8fbc8f",
  "darkslateblue": "#483d8b",
  "darkslategray": "#2f4f4f",
  "darkslategrey": "#2f4f4f",
  "darkturquoise": "#00ced1",
  "darkviolet": "#9400d3",
  "deeppink": "#ff1493",
  "deepskyblue": "#00bfff",
  "dimgray": "#696969",
  "dimgrey": "#696969",
  "dodgerblue": "#1e90ff",
  "firebrick": "#b22222",
  "floralwhite": "#fffaf0",
  "forestgreen": "#228b22",
  "fuchsia": "#ff00ff",
  "gainsboro": "#dcdcdc",
  "ghostwhite": "#f8f8ff",
  "goldenrod": "#daa520",
  "gold": "#ffd700",
  "gray": "#808080",
  "green": "#008000",
  "greenyellow": "#adff2f",
  "grey": "#808080",
  "honeydew": "#f0fff0",
  "hotpink": "#ff69b4",
  "indianred": "#cd5c5c",
  "indigo": "#4b0082",
  "ivory": "#fffff0",
  "khaki": "#f0e68c",
  "lavenderblush": "#fff0f5",
  "lavender": "#e6e6fa",
  "lawngreen": "#7cfc00",
  "lemonchiffon": "#fffacd",
  "lightblue": "#add8e6",
  "lightcoral": "#f08080",
  "lightcyan": "#e0ffff",
  "lightgoldenrodyellow": "#fafad2",
  "lightgray": "#d3d3d3",
  "lightgreen": "#90ee90",
  "lightgrey": "#d3d3d3",
  "lightpink": "#ffb6c1",
  "lightsalmon": "#ffa07a",
  "lightseagreen": "#20b2aa",
  "lightskyblue": "#87cefa",
  "lightslategray": "#778899",
  "lightslategrey": "#778899",
  "lightsteelblue": "#b0c4de",
  "lightyellow": "#ffffe0",
  "lime": "#00ff00",
  "limegreen": "#32cd32",
  "linen": "#faf0e6",
  "magenta": "#ff00ff",
  "maroon": "#800000",
  "mediumaquamarine": "#66cdaa",
  "mediumblue": "#0000cd",
  "mediumorchid": "#ba55d3",
  "mediumpurple": "#9370db",
  "mediumseagreen": "#3cb371",
  "mediumslateblue": "#7b68ee",
  "mediumspringgreen": "#00fa9a",
  "mediumturquoise": "#48d1cc",
  "mediumvioletred": "#c71585",
  "midnightblue": "#191970",
  "mintcream": "#f5fffa",
  "mistyrose": "#ffe4e1",
  "moccasin": "#ffe4b5",
  "navajowhite": "#ffdead",
  "navy": "#000080",
  "oldlace": "#fdf5e6",
  "olive": "#808000",
  "olivedrab": "#6b8e23",
  "orange": "#ffa500",
  "orangered": "#ff4500",
  "orchid": "#da70d6",
  "palegoldenrod": "#eee8aa",
  "palegreen": "#98fb98",
  "paleturquoise": "#afeeee",
  "palevioletred": "#db7093",
  "papayawhip": "#ffefd5",
  "peachpuff": "#ffdab9",
  "peru": "#cd853f",
  "pink": "#ffc0cb",
  "plum": "#dda0dd",
  "powderblue": "#b0e0e6",
  "purple": "#800080",
  "rebeccapurple": "#663399",
  "red": "#ff0000",
  "rosybrown": "#bc8f8f",
  "royalblue": "#4169e1",
  "saddlebrown": "#8b4513",
  "salmon": "#fa8072",
  "sandybrown": "#f4a460",
  "seagreen": "#2e8b57",
  "seashell": "#fff5ee",
  "sienna": "#a0522d",
  "silver": "#c0c0c0",
  "skyblue": "#87ceeb",
  "slateblue": "#6a5acd",
  "slategray": "#708090",
  "slategrey": "#708090",
  "snow": "#fffafa",
  "springgreen": "#00ff7f",
  "steelblue": "#4682b4",
  "tan": "#d2b48c",
  "teal": "#008080",
  "thistle": "#d8bfd8",
  "tomato": "#ff6347",
  "turquoise": "#40e0d0",
  "violet": "#ee82ee",
  "wheat": "#f5deb3",
  "white": "#ffffff",
  "whitesmoke": "#f5f5f5",
  "yellow": "#ffff00",
  "yellowgreen": "#9acd32"
}

colors = []
for i in color_code:
    colors.append(color_code[i])
    
#fill_color = ['red','green','blue','yellow','silver','black']
fill_color = colors

In [114]:
# Apply color for each route
for i in range(len(feature_col['features'])):
    feature_col['features'][i]['properties']['stroke'] = fill_color[i]
    feature_col['features'][i]['properties']['stroke-width'] = 8

In [115]:
## Try to think about how to get MultiLineString from Webscraping but I think it is really difficult. Remember that each point is not continous
### There are lines that are continous with points but remember that lines are connected. So route 77, for examples might have multiple lines (3? 5?) with multiple points in each line
### Therefore, you really have to just cut for each PolyLine coordinates and consider each PolyLine as different coordinates (total 2 coordinates with multiple coords in each)
### Refer to geojson.io randering on MultiStringLine to see how it works

### Thankfully, we have API from Miami dade so use that for now. Maybe I don't need to do web scraping for all of these routes. I think OpenHub API keeps updating anyway.

# Get StopInformation from API

In [116]:
import urllib.request, json 
with urllib.request.urlopen("https://opendata.arcgis.com/datasets/021adadcf6854f59852ff4652ad90c11_0.geojson") as url:
    stops_df = json.loads(url.read().decode())
    #print(route_df)

In [117]:
from pandas.io.json import json_normalize
stop_api = json_normalize(stops_df['features'])

In [118]:
stop_api[stop_api['properties.STOPID'] == 10378]

,geometry.coordinates,geometry.type,properties.ACCESSIBLE,properties.BENCHES,properties.CROSS_ST,properties.DIRECTION,properties.FID,properties.LAT,properties.LON,properties.MAIN_ST,properties.POINT_X,properties.POINT_Y,properties.SHELTERS,properties.STOPID,properties.STOP_CORNR,properties.TRASH_CAN,properties.UPDTE_DATE,type
7849,"[-80.21202569199994, 25.922776722000037]",Point,Y,6,(East lot),E,7850,25.92277,-80.21202,Golden Glades P&R,915151.948,578400.31,0,10378,T,0,10/17/2016 12:1,Feature


In [119]:
stop_api[stop_api['properties.MAIN_ST'] == 'Golden Glades P&R']

,geometry.coordinates,geometry.type,properties.ACCESSIBLE,properties.BENCHES,properties.CROSS_ST,properties.DIRECTION,properties.FID,properties.LAT,properties.LON,properties.MAIN_ST,properties.POINT_X,properties.POINT_Y,properties.SHELTERS,properties.STOPID,properties.STOP_CORNR,properties.TRASH_CAN,properties.UPDTE_DATE,type
1,"[-80.21506969099994, 25.92080072300007]",Point,N,2,(West lot),T,2,25.92080,-80.21507,Golden Glades P&R,914155.736,577676.084,1,156,T,2,9/20/2016 13:13,Feature
7849,"[-80.21202569199994, 25.922776722000037]",Point,Y,6,(East lot),E,7850,25.92277,-80.21202,Golden Glades P&R,915151.948,578400.310,0,10378,T,0,10/17/2016 12:1,Feature


In [120]:
len(full_df['StopID'].unique())

7536

In [121]:
# Note that API provides more stops tan from web-scraping but it means 480 stops are currently not available in real-time
len(stop_api['properties.STOPID'])

8000

In [122]:
stop_api['properties.STOPID'] = stop_api['properties.STOPID'].astype('int')
full_df['StopID'] = full_df['StopID'].astype('int')

In [123]:
full_df['RouteID'] = full_df['RouteID'].astype('int')

# Merge StopInfo from API and webscraping to get StopID/RouteID from webscraping

In [124]:
# merged_df = full_df
merged_df = stop_api.merge(full_df, left_on = 'properties.STOPID', right_on = 'StopID')
# if stop_api doesn't have some of the stops from full_df and vice versa, just use full_df and don't merge at all.
# Hmm, I think I noticed that stops that exist in full_df don't exist in stop_api.. figure out why

In [125]:
merged_df.columns
# Maybe use Properties.LAT/LON from stop_api instead of StopLat/Long from full_df..?

Index(['geometry.coordinates', 'geometry.type', 'properties.ACCESSIBLE',
       'properties.BENCHES', 'properties.CROSS_ST', 'properties.DIRECTION',
       'properties.FID', 'properties.LAT', 'properties.LON',
       'properties.MAIN_ST', 'properties.POINT_X', 'properties.POINT_Y',
       'properties.SHELTERS', 'properties.STOPID', 'properties.STOP_CORNR',
       'properties.TRASH_CAN', 'properties.UPDTE_DATE', 'type', ' MarkerType',
       'Marker', 'StopID', 'StopName', 'StopNameURL', 'Sequence', 'RouteID',
       'Dir', 'StopLat', 'StopLong'],
      dtype='object')

In [126]:
final_df = merged_df[['properties.STOPID', 'StopName','RouteID','properties.DIRECTION','properties.LAT','properties.LON','StopLat','StopLong']]
final_df.columns = ['StopID', 'StopName','RouteID','Direction','Lat','Lon','StopLat','StopLong']
# final_df = merged_df[['StopID', 'StopName','RouteID','Dir','StopLat','StopLong']]

In [127]:
final_df['StopID'] = final_df['StopID'].astype('int64')
final_df['RouteID'] = final_df['RouteID'].astype('int64')
final_df['StopLat'] = final_df['StopLat'].astype('float')
final_df['StopLong'] = final_df['StopLong'].astype('float')
final_df['Lat'] = final_df['Lat'].astype('float')
final_df['Lon'] = final_df['Lon'].astype('float')

C:\Users\andy\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
C:\Users\andy\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\andy\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pyd

In [128]:
stop_polygon = final_df

In [129]:
len(stop_polygon.RouteID.unique())

120

In [130]:
#final_df.to_csv('route_stop_mdc.csv')

# Draw ploygon using coordinates

In [144]:
import numpy as np
import json
import geog
#import shapely
import shapely.geometry

def coord_to_polygon(stop_polygon):
    polygon_list = []
    for i in range(len(stop_polygon)):
        p = shapely.geometry.Point(stop_polygon[['StopLong','StopLat']].values[i].astype('float'))
        n_points = 20
        d = 10 # meters
        angles = np.linspace(0, 360, n_points)
        polygon = geog.propagate(p, angles, d)
    #    polygon_list.append(json.dumps(shapely.geometry.mapping(shapely.geometry.Polygon(polygon))))
        polygon_list.append(polygon.tolist())
        
    return polygon_list

In [145]:
# filtered_list = [11,51,77,277,119,120]   ## use this if you want just 6 routes
filtered_list = [11,51,77,277,119,120,112,9,27,3,8,22]   ## use this if you want just 6 routes + expanding routes
filtered_list = sorted(filtered_list)  ## same as above -- need this to sort the route in ascending order

# filtered_list = stop_polygon['RouteID'].unique().tolist()  ## use this if you want ALL routes
# filtered_list = routes_list ## use this if you want ALL routes

# filtered_list = sorted(filtered_list) ## need this to sort the route in ascending order
filtered_df = stop_polygon[stop_polygon['RouteID'].isin(filtered_list)] 

In [146]:
filtered_df = filtered_df.sort_values(by='RouteID')

In [147]:
polygons = coord_to_polygon(stop_polygon=filtered_df)

In [148]:
# polyList = []
# filtered_list = [11,51,77,277,119,120]

# for i in filtered_list:
#     filtered_df = stop_polygon[stop_polygon.RouteID == i]
#     polyList.append(coord_to_polygon(stop_polygon=filtered_df))

# #full_df = pd.concat(pdList)

# converting df into GeoJson coordinates

In [149]:
def df_to_geojson_point(df, properties, lat='latitude', lon='longitude'):
    geojson = {'type':'FeatureCollection', 'features':[]}
    for _, row in df.iterrows():
        feature = {'type':'Feature',
                   'properties':{},
                   'geometry':{'type':'Point',
                               'coordinates':[]}}
        feature['geometry']['coordinates'] = [row[lon],row[lat]]
        for prop in properties:
            feature['properties'][prop] = row[prop]
        geojson['features'].append(feature)
    return geojson

In [150]:
# test = df_to_geojson_point(stop_polygon, ['RouteID','StopName','Direction'], lat='StopLat', lon='StopLong')

In [151]:
# tt = json.dumps(test)

# converting df into GeoJson polygon

In [152]:
def df_to_geojson_polygon(df, properties, lat='latitude', lon='longitude'):
    geojson = {'type':'FeatureCollection', 'features':[]}
    for _, row in df.iterrows():
        feature = {'type':'Feature',
                   'properties':{},
                   'geometry':{'type':'Polygon',
                               'coordinates':[]}}
        feature['geometry']['coordinates'] = [[row[lon],row[lat]]]
        for prop in properties:
            feature['properties'][prop] = row[prop]
        geojson['features'].append(feature)
    return geojson

In [153]:
# Apply fill color for each route differently
for i,j in zip(filtered_list,fill_color):
    filtered_df.loc[filtered_df['RouteID'] == i,'fill'] = j


In [154]:
#test2 = df_to_geojson_polygon(filtered_df, ['RouteID','StopName','Direction', 'StopID', 'fill'], lat='StopLat', lon='StopLong')
##Without fill just to make 1826 map look prettier
test2 = df_to_geojson_polygon(filtered_df, ['RouteID','StopName','Direction', 'StopID'], lat='StopLat', lon='StopLong')
# test2 = df_to_geojson_polygon(filtered_df, ['RouteID','StopName','StopID', 'fill'], lat='StopLat', lon='StopLong')

In [155]:
#test2['features'][0]['geometry']['type'] = 'Polygon'
for i in range(len(polygons)):
    test2['features'][i]['geometry']['coordinates'] = [polygons[i]]

In [156]:
# import json
# with open('6routes.geojson', 'w', encoding='utf-8') as f: # or, maybe use .geojson ?
#     json.dump(test2, f, ensure_ascii=False, indent=4) # Hmmm.. didn't know dump and dumps are different! Maybe I should do dumps instead in order to wrap with double colons "" instead of ''

In [157]:
import json
t = json.dumps(feature_col).replace('"[','[').replace('"]',']').replace(']"',']')

with open('routes_webscraping.json', 'w') as f: # or, maybe use .geojson ?
    f.write(t)

In [158]:
route_web = json.loads(t)

# Construct polyline

In [159]:
# tt = geojson

# for i in range(len(tt['features'])):
#     for j in range(len(tt['features'][i]['geometry']['coordinates'])):
#         for k in range(len(tt['features'][i]['geometry']['coordinates'][j])):
#             for l in range(len(tt['features'][i]['geometry']['coordinates'][j][k])):   
#                 tt['features'][i]['geometry']['coordinates'][j][k][l] = float(tt['features'][i]['geometry']['coordinates'][j][k][l])

In [160]:
# import shapely
# from shapely.geometry import MultiLineString

In [161]:
# for i in range(len(tt['features'])):
#     try:
#         print(MultiLineString(tt['features'][i]['geometry']['coordinates']))
#     except:
#         print('error')

In [162]:
# for i in range(len(geojson['features'])):
#     try:
#         print(MultiLineString(geojson['features'][i]['geometry']['coordinates']))
#     except:
#         print('error')

In [163]:
# for i in range(len(geojson['features'])):
#     MultiLineString(geojson['features'][100]['geometry']['coordinates'])

In [164]:
# for i in range(len(geojson['features'])):
#     print(i,len(geojson['features'][i]['geometry']['coordinates']))

In [165]:
# MultiLineString(geojson['features'][80]['geometry']['coordinates'])

In [176]:
# import geopandas as gpd
# test3 = gpd.GeoDataFrame.from_features(route_web)

In [ ]:
# d = MultiLineString(geojson['features'][80]['geometry']['coordinates'])  ## Change this into value and put it into column.. and then create GeoPandas features dataframe??

In [ ]:
# MultiLineString(geojson['features'][80]['geometry']['coordinates']).buffer(0.5)
## So I guess I just need to manually convert this back to GeoPandas format! Just like the one on the left!

In [ ]:
# gpd.GeoDataFrame(geojson['features'])

In [ ]:
# MultiLineString(geojson['features'][0]['geometry']['coordinates'])

# find a way to convert this to just value.. something like below

In [ ]:
# e = []
# for i in range(len(geojson['features'])):
#     try:
#         e.append(MultiLineString(geojson['features'][i]['geometry']['coordinates']))
#     except:
#         print(i,'error')
    
# if below doesn't work, find a way to get printed version of MultiLiestring and put that into e!

In [ ]:
# with open('error_55.geojson','w') as f:
#     json.dump(geojson['features'][53], f, ensure_ascii=False, indent=4)

In [ ]:
# MultiLineString(route_web['features'][53]['geometry']['coordinates'])

In [ ]:
# gpd.GeoSeries(e[0].buffer(0.01)).to_json()  # Great! Use for loop to make column for each features and coordinates and make this as dataframe column just like the one in the left .ipyn file!

In [ ]:
# print(MultiLineString(geojson['features'][0]['geometry']['coordinates']))

In [ ]:
test3.crs = {'init': 'epsg:4326'}
#test3.crs = {'init': 'epsg:3174'}

In [ ]:
test3.crs

In [ ]:
cpr_gdf = test3.to_crs({'init': 'epsg:3174'})  # convert degrees into meters
buffer_length_in_meters = (1 * 100) * 1.60934   # convert into 1 (1000 m * 1.60934) mile from meters
cpr_gdf['geometry'] = cpr_gdf.geometry.buffer(buffer_length_in_meters)

In [ ]:
cpr_gdf = cpr_gdf.to_crs({'init': 'epsg:4326'})  # convert geometry with 1 mile buffer back to degrees

In [ ]:
cpr_gdf.geometry[0]

In [ ]:
#line_poly = gpd.GeoSeries(test3.geometry.buffer(0.003)).to_json()   # thicker
#line_poly = gpd.GeoSeries(test3.geometry.buffer(0.0013)).to_json() # slim (ideal)
#line_poly = gpd.GeoSeries(test3.geometry.buffer(0.0001)).to_json() # request
line_poly = gpd.GeoSeries(test3.geometry.buffer(0.00005)).to_json() # request2 --- this is the one we used for off-peak (correct one)
#line_poly = gpd.GeoSeries(test3.geometry.buffer(0.02)).to_json()  # very thick

# line_poly = gpd.GeoSeries(cpr_gdf.geometry.buffer(5280)).to_json() # 1 mile? (don't use this wrong)

# line_poly = gpd.GeoSeries(cpr_gdf.geometry).to_json() # 1 mile (we used this for Kai)
line_poly = json.loads(line_poly)

In [165]:
#test3['LINEABBR'] = test3['LINEABBR'].astype('int')
test3['RouteID'] = test3['RouteID'].astype('int')

In [166]:
# only select routeIds from filtered_list
line_poly_route_ids = test3['RouteID'].unique().tolist()


# for i in range(len(filtered_list)):
#     for j in filtered_list:
#         line_poly['features'][i]['properties']['routes'] = j
        
# for i,j in enumerate(filtered_list):
#     line_poly['features'][i]['properties']['routes'] = j
    
for i,j in enumerate(line_poly_route_ids):
    line_poly['features'][i]['properties']['routes'] = j

# Merge Stops with PolyRoutes and line

In [167]:
def append_stops(stop_df):
    for i in range(len(stop_df['features'])):
        route_web['features'].append(stop_df['features'][i])

In [168]:
append_stops(test2)  # append each feature in polygon stops  -- for 1 mile stop, you don't need stops. don't add them (2020-09-14)

In [169]:
append_stops(line_poly) # append each feature in polyline

# Graph in folium with polyline and routes only for now

In [174]:
# import folium
# from IPython.display import display

#gjson = hidro.to_crs(epsg='4326').to_json()
# mapa = folium.Map([25.67, -81.866667],
#                   zoom_start=4,
#                   tiles='cartodbpositron')
# looks like this gjson is working by each route only (not working if I put everything in one shot)
# Maybe I should do the same thing for stops in polygon..? by stops for each route..?
# Also, why can't I change the color for each route..? Find a reason!
# Lastly, find if you can graph everything in just 1 geojson fil (it worked in local machine but why not in Jupyter..? Weird)

#2020-07-01
# For map, maybe I can add parameters that users choose in the map -- diameter for stops and buffer size for route (polygon size) , 
## routeID, users (meaning show all the markers/icons that are within the stops) who are within Stops, users who are within Route, users who are not within stops, users who are not within Route and etc
## Also, if I can, try to add Bus image as icon/marker for bus stops (at the center of the diameter for stop polygon)

# points = folium.features.GeoJson(gjson6)

# mapa.add_child(points)

# display(mapa)

In [ ]:
# Export RouteLines and Polygons around it only
# with open('route120_polylines_routes.json', 'w', encoding='utf-8') as f: # or, maybe use .geojson ?
#     json.dump(route_web, f, ensure_ascii=False, indent=4)

In [ ]:
# import github3
# ghapi = github3.login('andy.sy.hwang@gmail.com', password='Qlqjs1123!') #github3.GitHub() 
# files = {'route120_polylines_routes.json': {'content': json.dumps(route_web)}} #json.dumps(geojson)
# gist = ghapi.create_gist('', files, public=False)

In [ ]:
#WRONG!!
# final = test2
# final['features'].append(geojson['features'])
# final['features'].append(line_poly['features'])

### Export the result into geojson/json

In [172]:
name = 'polylines_6routes_expandingroutes_includingstops_buffer(0.00005).geojson'
with open(name, 'w', encoding='utf-8') as f: # or, maybe use .geojson ?
    json.dump(route_web, f, ensure_ascii=False, indent=4)

In [478]:
# Export polygon stops only
with open('MiamiRoute1mile.geojson', 'w', encoding='utf-8') as f: # or, maybe use .geojson ?
    json.dump(route_web, f, ensure_ascii=False, indent=4)   

In [168]:
# Export polylines only
with open('polylines_6routes_expandingroutes buffer(0.00005).json', 'w', encoding='utf-8') as f: # or, maybe use .geojson ?
    json.dump(line_poly, f, ensure_ascii=False, indent=4)

# Rander map in gist

In [173]:
import github3
ghapi = github3.login('andy.sy.hwang@gmail.com', password='') #github3.GitHub() 
files = {name: {'content': json.dumps(route_web)}} #json.dumps(geojson)
gist = ghapi.create_gist('', files, public=False)

In [195]:
import github3
ghapi = github3.login('andy.sy.hwang@gmail.com', password='') #github3.GitHub() 
files = {'all_realtime_routes_polygon_only_1826.json': {'content': json.dumps(route_web)}} #json.dumps(geojson)
gist = ghapi.create_gist('', files, public=False)

In [169]:
import github3
ghapi = github3.login('andy.sy.hwang@gmail.com', password='') #github3.GitHub() 
files = {'polylines_6routes_expandingroutes buffer(0.00005).json': {'content': json.dumps(line_poly)}} #json.dumps(geojson)  #json.dumps(route_web) if you want multil line plus polygon
gist = ghapi.create_gist('', files, public=False)

In [ ]:
import geojsonio
geojsonio.display(json.dumps(route_web))


In [ ]:
# import folium
# from IPython.display import display

# #gjson = hidro.to_crs(epsg='4326').to_json()
# mapa = folium.Map([25.67, -81.866667],
#                   zoom_start=4,
#                   tiles='cartodbpositron')
# # looks like this gjson is working by each route only (not working if I put everything in one shot)
# # Maybe I should do the same thing for stops in polygon..? by stops for each route..?
# # Also, why can't I change the color for each route..? Find a reason!
# # Lastly, find if you can graph everything in just 1 geojson fil (it worked in local machine but why not in Jupyter..? Weird)

# #2020-07-01
# # For map, maybe I can add parameters that users choose in the map -- diameter for stops and buffer size for route (polygon size) , 
# ## routeID, users (meaning show all the markers/icons that are within the stops) who are within Stops, users who are within Route, users who are not within stops, users who are not within Route and etc
# ## Also, if I can, try to add Bus image as icon/marker for bus stops (at the center of the diameter for stop polygon)

# points = folium.features.GeoJson(route_web)

# mapa.add_child(points)

# display(mapa)

# Rander map in ipyleaflet

In [170]:
# from ipyleaflet import GeoJSON
# from ipyleaflet import Map, basemaps

# geo = GeoJSON(data=route_web)
# # First is latitude and second is longitude; both in degrees
# center = (25.768947999991482, -80.35031706845287)

# map = Map(center=center, interpolation='nearest', basemap=basemaps.Stamen.Terrain)

# map.add_layer(geo)



In [ ]:
from ipywidgets.embed import embed_minimal_html

embed_minimal_html('carpentries_instructors_basic.html', views=[map], title='The Carpentries Instructors')

# Rander map in GoogleMap

In [68]:
# import gmaps
# import gmaps.datasets
# gmaps.configure(api_key="AI...") # Your Google API key

# # load a Numpy array of (latitude, longitude) pairs
# locations = gmaps.datasets.load_dataset("taxi_rides")

# fig = gmaps.figure()
# fig.add_layer(gmaps.heatmap_layer(locations))
# fig